In [ ]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
import itertools
from matplotlib import pyplot as plt
from datetime import datetime, timedelta

path = '#'

In [ ]:
## import survey data set

infile = open(path+'umfrage2020only.pkl','rb')
umfrage = pickle.load(infile)
infile.close()

# Deutschnote: bereinigen
umfrage['deutschnote'] = umfrage['deutschnote'].replace(['Es gab keine Noten', np.nan],0)
umfrage['deutschnote'] = umfrage['deutschnote'].astype('int')
# Erstschrift: deutsch=1, nicht deutsch=0
umfrage['ErstSchrift'] = umfrage['ErstSchrift'].replace([np.nan,'Russisch','Serbisch','Englisch','Italienisch','Vietnamesisch','Persisch','Kroatisch','Polnisch','Kurdisch','Arabisch','Griechisch','Spanisch','Niederl&auml;ndisch','T&uuml;rkisch','eine andere, und zwar...'],0)
umfrage['ErstSchrift'] = umfrage['ErstSchrift'].replace(['Deutsch'],1)
# Eigsprache: deutsch=1, nicht deutsch=0
umfrage['eigSprache'] = umfrage['eigSprache'].replace([np.nan,'Russisch','Serbisch','Englisch','Italienisch','Vietnamesisch','Persisch','Kroatisch','Polnisch','Kurdisch','Arabisch','Griechisch','Spanisch','Niederl&auml;ndisch','T&uuml;rkisch','eine andere, und zwar...'],0)
umfrage['eigSprache'] = umfrage['eigSprache'].replace(['Deutsch'],1)
# Sprache mutter: deutsch = 1, nicht deutsch = 0
umfrage['SpracheMutter'] = umfrage['SpracheMutter'].replace([np.nan,'Russisch','Serbisch','Englisch','Italienisch','Vietnamesisch','Persisch','Kroatisch','Polnisch','Kurdisch','Arabisch','Griechisch','Spanisch','Niederl&auml;ndisch','T&uuml;rkisch','eine andere, und zwar...'],0)
umfrage['SpracheMutter'] = umfrage['SpracheMutter'].replace(['Deutsch'],1)
# Sprache vater: deutsch = 1, nicht deutsch = 0
umfrage['SpracheVater'] = umfrage['SpracheVater'].replace([np.nan,'Russisch','Serbisch','Englisch','Italienisch','Vietnamesisch','Persisch','Kroatisch','Polnisch','Kurdisch','Arabisch','Griechisch','Spanisch','Niederl&auml;ndisch','T&uuml;rkisch','eine andere, und zwar...'],0)
umfrage['SpracheVater'] = umfrage['SpracheVater'].replace(['Deutsch'],1)
#Abieltern: weißnicht = nan
umfrage['AbiEltern'] = umfrage['AbiEltern'].replace(['weissnicht'],np.nan)
#Buecher
umfrage['Buecher'] = umfrage['Buecher'].replace(['10'],0)
umfrage['Buecher'] = umfrage['Buecher'].replace(['50'],1)
umfrage['Buecher'] = umfrage['Buecher'].replace(['100'],2)
umfrage['Buecher'] = umfrage['Buecher'].replace(['200'],3)

relevant_cols = [  'SchreibenGern', 'SchreibenLeicht','deutschnote',
       'eigSprache', 'ErstSchrift', 'SpracheMutter', 'LesenGern',
       'SpracheVater', 'LesenLeicht']#'AbiEltern'

for i in relevant_cols:
    umfrage[i] = umfrage[i].astype('float32')
    umfrage[i] = umfrage[i].fillna(umfrage[i].mean())

umfrage = umfrage[['UserID','AbiEltern','Buecher','eigSprache']]

In [ ]:
## import preprocessed data
infile = open('../04_predictionUserHistory/01_data/final_data_preprocessed.pkl','rb')
import_file = pickle.load(infile)
infile.close()
smaller_features=import_file[['UserID','Erstloesung','Erfolg','Schwierigkeit','Wochentag','ist_Schulzeit','MehrfachFalsch','Testposition__FT', 'Testposition__nt', 'Testposition__pruefung',
       'Testposition__training', 'Testposition__version', 'Testposition__vt',
       'Testposition__zt', 'beendet', 'Fehler', 'HA__HA', 'HA__Self', 'HA__nt',
       'HA__vt', 'HA__zt', 'Klassenstufe', 'Jahredabei',
       'Sex__m', 'Sex__w']]

# keep only users who took part in survey
userIds = list(umfrage['UserID'])
smaller_features = smaller_features[smaller_features['UserID'].isin(userIds)]

In [ ]:
## import all sentences
infile = open(path+'saetze.pkl','rb')
saetze = pickle.load(infile)
infile.close()
saetze = saetze[['satzID','Schwierigkeit','Art','AufgabenID']]

#Art: only GK
options = ['GK'] 
saetze = saetze[saetze['Art'].isin(options)] 

#Aufgaben ID : all except 0 (0 is kompetenztests) 
options = ['0'] 
saetze = saetze[~saetze['AufgabenID'].isin(options)] 

Engineer historical learning data for each user who took part in the survey

In [ ]:
## create empty df with null values
# rows: all userIDs from preprocessed dataset
# columns: all satzIDs

userIDs = smaller_features[['UserID']]
userIDs = userIDs.drop_duplicates()
index_col = userIDs['UserID'].tolist()
satzList = saetze["satzID"].tolist()
df = pd.DataFrame(0, index =index_col,columns =satzList)

In [ ]:
## import xmlsaetze and xmlsaetze archiv to get historical data
infile = open(path+'xmlsaetze.pkl','rb')
xmlsaetze = pickle.load(infile)
infile.close()
xmlsaetze = xmlsaetze[['ID','UserID','UebungsID','SatzID','Erfolg','Datum']]

infile = open(path+'xmlsaetze_archiv.pkl','rb')
xmlsaetze_archiv = pickle.load(infile)
infile.close()
xmlsaetze_archiv = xmlsaetze_archiv[['ID','UserID','UebungsID','SatzID','Erfolg','Datum']]

xmlsaetze = xmlsaetze.append(xmlsaetze_archiv)
xmlsaetze_small = xmlsaetze[['UserID','SatzID','Erfolg','Datum']]

# filter satzID: only GK and not kompetenztests
options = saetze['satzID'].tolist()
xmlsaetze_small = xmlsaetze_small[xmlsaetze_small['SatzID'].isin(options)] 

# filter userID: only User which appear in preprocessed dataset
options = index_col
xmlsaetze_userid = xmlsaetze_small[xmlsaetze_small['UserID'].isin(options)] 

In [ ]:
## add column last_login
# not neccessary in production: in production we use t-3months 
# neccessary for training
last_login = xmlsaetze_userid[['UserID','Datum']].groupby(['UserID']).max()
last_login = pd.DataFrame(last_login.reset_index())
last_login = last_login.rename(columns={"Datum": "last_login"})

# merge last login 
xmlsaetze_userid_last_login= xmlsaetze_userid.merge(last_login, on='UserID', how='left')

In [ ]:
## iterate through xmlsaetze to finde historical data for each user
for index, row in xmlsaetze_userid_last_login.iterrows():
    user_cell = row['userid']
    satz_cell = row['satz']
    erfolg_cell = row['erfolg']
    datum_cell = row['datum']
    last_login_cell = row['last_login']

    dateOfExercise = datetime.fromisoformat(str(datum_cell)).date()
    dateOfLastLogin = datetime.fromisoformat(str(last_login_cell)).date()
    acceptedDate = dateOfLastLogin + pd.DateOffset(months=-3)# accepted date calculates the date of the last login minus 3 months

    #check if sentence and user is in df
    if satz_cell in df.columns and user_cell in df.index and dateOfExercise > acceptedDate:
        df.at[user_cell, satz_cell] = 1 if erfolg_cell == 1 else -1


In [ ]:
## join historical data to dataset and to survey
historical_join = df.reset_index()
historical_join = historical_join.rename(columns={"index": "UserID"})
historical_join.to_pickle('historicalJoin_3months.pkl')

dataset = pd.merge(smaller_features, historical_join, how='left', on='UserID')
dataset.to_pickle('smallSampleSet_3months.pkl')

df = pd.merge(dataset, umfrage, on='UserID')
df.to_pickle('preprocessed_umfrage.pkl')

In [ ]:
################### SUBGROUPING

# Abi eltern
df.AbiEltern = df.AbiEltern.astype('float')
df_abi = df[df.AbiEltern > 0]
df_keinAbi = df[df.AbiEltern ==0]
df_weissnicht = df[df.AbiEltern.isnull()]

print(len(df_abi.UserID.unique()))
print(len(df_keinAbi.UserID.unique()))
print(len(df_weissnicht.UserID.unique()))

#Gender
df_boys = df[df.Sex__m == 1]
df_girls = df[df.Sex__w == 1]

print(len(df_boys.UserID.unique()))
print(len(df_girls.UserID.unique()))

# Migration
df_deutsch = df[df.eigSprache == 1]
df_migration = df[df.eigSprache == 0]

print(len(df_deutsch.UserID.unique()))
print(len(df_migration.UserID.unique()))

# Anzahl Bücher
df['Buecher'] = df['Buecher'].replace({'10':0, '200':1})
df_buch0 = df[df.Buecher == 0.0]
df_buch1 = df[df.Buecher == 1]

print(len(df_buch0.UserID.unique()))
print(len(df_buch1.UserID.unique()))

In [ ]:
## save subgroups to pickle files
df_abi.to_pickle('double_df_abi.pkl')
df_keinAbi.to_pickle('double_df_keinAbi.pkl')
df_weissnicht.to_pickle('double_df_weissnicht.pkl')
df_boys.to_pickle('double_df_boys.pkl')
df_girls.to_pickle('double_df_girls.pkl')
df_deutsch.to_pickle('double_df_deutsch.pkl')
df_migration.to_pickle('double_df_migration.pkl')
df_buch0.to_pickle('double_df_buch0.pkl')
df_buch1.to_pickle('double_df_buch1.pkl')